# 🎲 VAE (Variational Autoencoder): Espaço Latente Probabilístico

**Tutorial de Espaço Latente - Notebook 3**

## 🎯 Objetivos
- Entender a diferença entre Autoencoder e VAE
- Compreender o truque de reparametrização
- Treinar um VAE no MNIST
- Gerar novas amostras
- Comparar com Autoencoder tradicional

## 🆚 Autoencoder vs VAE

### Autoencoder Tradicional
```
x → Encoder → z (ponto fixo) → Decoder → x'
```
- z é determinístico
- Espaço latente pode ter "buracos"

### VAE (Variational Autoencoder)
```
x → Encoder → (μ, σ²) → z ~ N(μ, σ²) → Decoder → x'
```
- z é uma **distribuição** (não um ponto)
- Aprende μ (média) e σ² (variância)
- Espaço latente é contínuo e suave
- Pode gerar novas amostras!

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from src.models.vae import VAE, vae_loss
from src.utils.data_loader import load_mnist
from src.utils.training import train_vae
from src.utils.visualization import (
    plot_vae_results,
    visualize_latent_space,
    plot_latent_grid,
    plot_training_history
)

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {DEVICE}")

## 📊 Carregando Dados

In [ ]:
train_loader, val_loader, test_loader = load_mnist(batch_size=128)
print(f"Data loaded: {len(train_loader.dataset)} train samples")

## 🏗️ Criando o VAE

In [ ]:
vae = VAE(
    input_dim=784,
    latent_dim=2,  # 2D para visualização
    hidden_dims=[512, 256]
)

print(vae)
print(f"\nParameters: {sum(p.numel() for p in vae.parameters()):,}")

## 🎓 Função de Perda do VAE

$$\mathcal{L} = \underbrace{\text{BCE}(x, x')}_{\text{Reconstruction}} + \beta \cdot \underbrace{\text{KL}(q(z|x) || p(z))}_{\text{Regularização}}$$

- **Reconstruction Loss**: Quão bem reconstruímos a imagem
- **KL Divergence**: Força z a seguir N(0,1)
- **β**: Peso do termo KL (β=1 para VAE padrão)

In [ ]:
# Demonstração da loss
data, _ = next(iter(train_loader))
data = data.view(-1, 784).to(DEVICE)
vae = vae.to(DEVICE)

# Forward pass
x_recon, mu, logvar, z = vae(data[:4])
loss_dict = vae_loss(x_recon, data[:4], mu, logvar, beta=1.0)

print("Loss components (before training):")
print(f"  Total: {loss_dict['total'].item():.2f}")
print(f"  Reconstruction: {loss_dict['reconstruction'].item():.2f}")
print(f"  KL Divergence: {loss_dict['kl'].item():.2f}")

## 🎓 Treinamento

In [ ]:
# Treina VAE
history = train_vae(
    model=vae,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=30,
    learning_rate=1e-3,
    beta=1.0,
    device=DEVICE,
    early_stopping_patience=7,
    verbose=True
)

# Plota histórico
plot_training_history(history)

## 📊 Visualização Completa dos Resultados

In [ ]:
# Overview completo
plot_vae_results(vae, test_loader, device=DEVICE)

## 🌌 Explorando o Manifold Latente

Uma das grandes vantagens do VAE: todo ponto do espaço latente gera uma imagem válida!

In [ ]:
# Grid 2D do espaço latente
plot_latent_grid(vae, n_samples=20, latent_range=(-3, 3), device=DEVICE)

## 🎨 Gerando Novas Amostras

VAE pode gerar dígitos totalmente novos!

In [ ]:
# Gera amostras do prior N(0,1)
n_samples = 16
samples = vae.sample(num_samples=n_samples, device=DEVICE)
samples = samples.view(-1, 28, 28).cpu()

fig, axes = plt.subplots(4, 4, figsize=(8, 8))
for i, ax in enumerate(axes.flat):
    ax.imshow(samples[i], cmap='gray')
    ax.axis('off')
plt.suptitle('Generated Digits (sampled from N(0,1))', fontweight='bold')
plt.tight_layout()
plt.show()

## 🔬 Comparação: VAE vs Autoencoder

In [ ]:
from src.models.autoencoder import Autoencoder

# Treina autoencoder para comparação
ae = Autoencoder(input_dim=784, latent_dim=2, hidden_dims=[512, 256, 128])

from src.utils.training import train_model
history_ae = train_model(
    model=ae,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=15,
    device=DEVICE,
    verbose=False
)

# Visualiza ambos os espaços latentes
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Autoencoder
ae.eval()
latents_ae = []
labels_ae = []
with torch.no_grad():
    for data, labels in test_loader:
        data = data.view(-1, 784).to(DEVICE)
        _, latent = ae(data)
        latents_ae.append(latent.cpu())
        labels_ae.append(labels)
latents_ae = torch.cat(latents_ae).numpy()[:1000]
labels_ae = torch.cat(labels_ae).numpy()[:1000]

ax1.scatter(latents_ae[:, 0], latents_ae[:, 1], c=labels_ae, cmap='tab10', alpha=0.5, s=5)
ax1.set_title('Autoencoder Latent Space', fontweight='bold', fontsize=14)
ax1.set_xlabel('z[0]')
ax1.set_ylabel('z[1]')
ax1.grid(True, alpha=0.3)

# VAE
vae.eval()
latents_vae = []
labels_vae = []
with torch.no_grad():
    for data, labels in test_loader:
        data = data.view(-1, 784).to(DEVICE)
        mu, _ = vae.encode(data)
        latents_vae.append(mu.cpu())
        labels_vae.append(labels)
latents_vae = torch.cat(latents_vae).numpy()[:1000]
labels_vae = torch.cat(labels_vae).numpy()[:1000]

scatter = ax2.scatter(latents_vae[:, 0], latents_vae[:, 1], c=labels_vae, cmap='tab10', alpha=0.5, s=5)
ax2.set_title('VAE Latent Space', fontweight='bold', fontsize=14)
ax2.set_xlabel('z[0]')
ax2.set_ylabel('z[1]')
ax2.grid(True, alpha=0.3)

plt.colorbar(scatter, ax=ax2, label='Digit')
plt.tight_layout()
plt.show()

print("\n📊 Observações:")
print("- VAE tem espaço mais 'suave' e contínuo")
print("- Autoencoder pode ter 'buracos' entre clusters")
print("- VAE force distribuição Gaussiana → melhor para geração")

## 📝 Resumo

✅ VAE aprende distribuição probabilística no espaço latente  
✅ Loss = Reconstruction + KL Divergence  
✅ Pode gerar novas amostras facilmente  
✅ Espaço latente mais contínuo que Autoencoder  
✅ Trade-off entre qualidade e regularização (controlado por β)  

---

## 🚀 Próximo Notebook

No **Notebook 04**, vamos explorar Beta-VAE e o conceito de disentanglement!

→→ `04_beta_vae_experimento.ipynb`